In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [2]:
from datetime import datetime
import io
import itertools
from packaging import version

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

Init Plugin
Init Graph Optimizer
Init Kernel
TensorFlow version:  2.5.0


In [3]:
# Download the data. The data is already divided into train and test.
# The labels are integers representing classes.
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = \
    fashion_mnist.load_data()

# Names of the integer classes, i.e., 0 -> T-short/top, 1 -> Trouser, etc.
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

train_images = train_images.reshape((train_images.shape[0], 28, 28, 1))
test_images = test_images.reshape((test_images.shape[0], 28, 28, 1))
# one hot encode target values
train_labels = tf.keras.utils.to_categorical(train_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)

train_norm = train_images.astype('float32')
test_norm = test_images.astype('float32')
train_norm = train_norm / 255.0
test_norm = test_norm / 255.0

In [4]:

model = keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28,28,1))) 
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.summary()

model.compile(
    optimizer='adam', 
    #loss='sparse_categorical_crossentropy',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

2022-03-21 10:33:48.673557: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-21 10:33:48.673654: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 64)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        8224      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 32)          0         
_________________________________________________________________
flatten (Flatten)         

In [5]:
def plot_to_image(figure):
  """Converts the matplotlib plot specified by 'figure' to a PNG image and
  returns it. The supplied figure is closed and inaccessible after this call."""
  # Save the plot to a PNG in memory.
  buf = io.BytesIO()
  plt.savefig(buf, format='png')
  # Closing the figure prevents it from being displayed directly inside
  # the notebook.
  plt.close(figure)
  buf.seek(0)
  # Convert PNG buffer to TF image
  image = tf.image.decode_png(buf.getvalue(), channels=4)
  # Add the batch dimension
  image = tf.expand_dims(image, 0)
  return image

def image_grid():
  """Return a 5x5 grid of the MNIST images as a matplotlib figure."""
  # Create a figure to contain the plot.
  figure = plt.figure(figsize=(10,10))
  for i in range(25):
    # Start next subplot.
    plt.subplot(5, 5, i + 1, title=class_names[train_labels[i]])
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
  
  return figure

In [6]:
def plot_confusion_matrix(cm, class_names):
  """
  Returns a matplotlib figure containing the plotted confusion matrix.

  Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
  """
  figure = plt.figure(figsize=(8, 8))
  plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
  plt.title("Confusion matrix")
  plt.colorbar()
  tick_marks = np.arange(len(class_names))
  plt.xticks(tick_marks, class_names, rotation=45)
  plt.yticks(tick_marks, class_names)

  # Compute the labels from the normalized confusion matrix.
  labels = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

  # Use white text if squares are dark; otherwise black.
  threshold = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    color = "white" if cm[i, j] > threshold else "black"
    plt.text(j, i, labels[i, j], horizontalalignment="center", color=color)

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  return figure

In [7]:
def log_confusion_matrix(epoch, logs):
  # Use the model to predict the values from the validation dataset.
  test_pred_raw = model.predict(test_images)
  test_pred = np.argmax(test_pred_raw, axis=1)
  test_lbl = np.argmax(test_labels, axis=1)  

  # Calculate the confusion matrix.
  cm = sklearn.metrics.confusion_matrix(test_lbl, test_pred)
  # Log the confusion matrix as an image summary.
  figure = plot_confusion_matrix(cm, class_names=class_names)
  cm_image = plot_to_image(figure)

  # Log the confusion matrix as an image summary.
  with file_writer_cm.as_default():
    tf.summary.image("Confusion Matrix", cm_image, step=epoch)

# Define the per-epoch callback.
cm_callback = keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [8]:
# Clear out prior logging data.
!rm -rf logs/image

logdir = "logs/image/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# Define the basic TensorBoard callback.
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

2022-03-21 10:34:05.706133: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2022-03-21 10:34:05.706146: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2022-03-21 10:34:05.706229: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.


In [9]:
# Start TensorBoard.
%tensorboard --logdir logs/image

In [16]:
#print(train_norm.shape, train_labels.shape)
# Train the classifier.
model.fit(
    train_norm,
    train_labels,
    batch_size=64,
    epochs=15,
    verbose=1, # Suppress chatty output
    callbacks=[tensorboard_callback, cm_callback],
    validation_data=(test_norm,test_labels)
)

Epoch 1/15
 14/938 [..............................] - ETA: 11s - loss: 0.1929 - accuracy: 0.9230

2021-09-16 14:44:42.109820: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-09-16 14:44:42.109834: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-09-16 14:44:42.126458: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-09-16 14:44:42.126973: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-09-16 14:44:42.127635: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/image/20210916-144430/train/plugins/profile/2021_09_16_14_44_42
2021-09-16 14:44:42.128009: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to logs/image/20210916-144430/train/plugins/profile/2021_09_16_14_44_42/AD134914-MAC.trace.json.gz
2021-09-16 14:44:42.128558: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: logs/image/20210916-144430/train/p

938/938 [==============================] - 12s 13ms/step - loss: 0.2069 - accuracy: 0.9239 - val_loss: 0.2194 - val_accuracy: 0.9235
Epoch 2/15
938/938 [==============================] - 12s 13ms/step - loss: 0.2085 - accuracy: 0.9227 - val_loss: 0.2229 - val_accuracy: 0.9213
Epoch 3/15
938/938 [==============================] - 12s 13ms/step - loss: 0.2060 - accuracy: 0.9219 - val_loss: 0.2199 - val_accuracy: 0.9219
Epoch 4/15
938/938 [==============================] - 12s 13ms/step - loss: 0.2005 - accuracy: 0.9250 - val_loss: 0.2263 - val_accuracy: 0.9208
Epoch 5/15
938/938 [==============================] - 12s 13ms/step - loss: 0.2018 - accuracy: 0.9254 - val_loss: 0.2184 - val_accuracy: 0.9232
Epoch 6/15
938/938 [==============================] - 12s 13ms/step - loss: 0.2004 - accuracy: 0.9243 - val_loss: 0.2149 - val_accuracy: 0.9230
Epoch 7/15
938/938 [==============================] - 12s 13ms/step - loss: 0.1977 - accuracy: 0.9251 - val_loss: 0.2231 - val_accuracy: 0.9207
Epo

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/images_accuracy.png?raw=1"/> -->

<!-- <img class="tfo-display-only-on-site" src="https://github.com/tensorflow/tensorboard/blob/master/docs/images/images_cm.png?raw=1"/> -->